In [ ]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; plat

In [ ]:
!wget https://object.pouta.csc.fi/OPUS-NLLB/v1/moses/ru-tt.txt.zip -O NLLB.zip
!unzip NLLB.zip -d NLLB

!wget https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2024/moses/ru-tt.txt.zip -O OpenSubtitles.zip
!unzip OpenSubtitles.zip -d OpenSubtitles

!wget https://object.pouta.csc.fi/OPUS-wikimedia/v20230407/moses/ru-tt.txt.zip -O wikimedia.zip
!unzip wikimedia.zip -d wikimedia

!wget https://object.pouta.csc.fi/OPUS-Tanzil/v1/moses/ru-tt.txt.zip -O Tanzil.zip
!unzip Tanzil.zip -d Tanzil

!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v2023-04-12/moses/ru-tt.txt.zip -O Tatoeba.zip
!unzip Tatoeba.zip -d Tatoeba


--2025-06-05 10:51:44--  https://object.pouta.csc.fi/OPUS-NLLB/v1/moses/ru-tt.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49301329 (47M) [application/zip]
Saving to: ‘NLLB.zip’

NLLB.zip            100%[===================>]  47.02M  13.8MB/s    in 3.6s    

2025-06-05 10:51:48 (13.2 MB/s) - ‘NLLB.zip’ saved [49301329/49301329]

Archive:  NLLB.zip
  inflating: NLLB/README             
  inflating: NLLB/LICENSE            
  inflating: NLLB/NLLB.ru-tt.ru      
  inflating: NLLB/NLLB.ru-tt.tt      
  inflating: NLLB/NLLB.ru-tt.scores  
--2025-06-05 10:51:51--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2024/moses/ru-tt.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTT

In [ ]:
from datasets import load_dataset
import pandas as pd

datasets = []


datasets.append(
    load_dataset('AigizK/tatar-russian-parallel-corpora', split="train").to_pandas().rename(columns={"rus": "ru"})
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/543 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/29.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/161831 [00:00<?, ? examples/s]

In [ ]:
def extract_data(file):
  with open(file + '.ru') as f:
    dataset_ru = [line.rstrip() for line in f]
  with open(file + '.tt') as f:
    dataset_tat = [line.rstrip() for line in f]
  return pd.DataFrame({'tat': dataset_tat, 'ru': dataset_ru}, columns=['tat', 'ru'])

In [ ]:
files = ['OpenSubtitles/OpenSubtitles.ru-tt', 'wikimedia/wikimedia.ru-tt', 'Tanzil/Tanzil.ru-tt', 'Tatoeba/Tatoeba.ru-tt']

for file in files:
  datasets.append(extract_data(file))

In [ ]:
import re

def clean_text(text):
    if not isinstance(text, str):
        return ""

    # Удаляет всё между (), включая скобки
    text = re.sub(r'\([^)]*\)', '', text)
    # Удаляет всё между [], включая скобки
    text = re.sub(r'\[[^]]*\]', '', text)

    text = re.sub(r'<.*?>', ' ', text)                      # HTML
    text = re.sub(r'\S+@\S+', ' ', text)                    # Emails
    text = re.sub(r'http\S+|www\.\S+', ' ', text)           # URLs
    text = re.sub(r'[\U00010000-\U0010ffff]', '', text)     # Emojis
    text = re.sub(r'[^\s.!?,0-9а-яА-ЯёЁүҮөӨңҢҗҖһҺәӘ-]', ' ', text)  # спецсимволы
    text = re.sub(r'\s+', ' ', text)                        # множественные пробелы
    return text.strip()

def normalize_text(text):
    text = text.lower()                     # нижний регистр
    text = re.sub(r'\s+', ' ', text)        # множественные пробелы
    return text.strip()

In [ ]:
for i in range(len(datasets)):
    datasets[i] = datasets[i][~datasets[i]['ru'].str.contains(r'[A-Za-z]', na=False)]
    datasets[i] = datasets[i][~datasets[i]['tat'].str.contains(r'[A-Za-z]', na=False)]
    datasets[i]['ru'] = datasets[i]['ru'].astype(str).apply(clean_text).apply(normalize_text)
    datasets[i]['tat'] = datasets[i]['tat'].astype(str).apply(clean_text).apply(normalize_text)
    datasets[i].dropna(inplace=True)
    datasets[i].drop_duplicates(inplace=True)
    datasets[i] = datasets[i][(datasets[i]['ru'].str.strip() != '') & (datasets[i]['tat'].str.strip() != '')]
    datasets[i] = datasets[i][
      (datasets[i]["ru"].str.split().str.len().between(1, 25)) &
      (datasets[i]["tat"].str.split().str.len().between(1, 25))
    ]


LETTERS_SET = set("абвгдеёжзийклмнопрстуфхцчшщъыьэюяәөүҗңһҺӘӨҮҢҖАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ")

def fast_check(text):
    if not isinstance(text, str):
        return False
    count = 0
    for char in text:
        if char in LETTERS_SET:
            count += 1
            if count >= 2:
                return True
    return False

In [ ]:
corpus_par = pd.concat(datasets)

In [ ]:
corpus_par = corpus_par[
    corpus_par['tat'].apply(fast_check) &
    corpus_par['ru'].apply(fast_check)
]

In [ ]:
corpus_par

,tat,ru
0,рэп-баттл ярдәмендә фигыльләрне өйрәнәбез .,учим глаголы при помощи рэп-баттла .
1,россия өчен укытучы проектында катнашучылар ав...,участники проекта учитель для россии о сельски...
2,2015 елда берничә төбәктә россия өчен укытучы ...,в 2015 году в нескольких регионах стартовала п...
3,нәкъ шундый ук проектлар күп кенә илләрдә һинд...,точно такие же проекты существуют во многих ст...
4,болар барысы да шул исәптән мәктәпкә мөнәсәбәт...,"все это придумано в том числе и для того, чтоб..."
...,...,...
3296,бу минем өчен җитәрлек түгел.,для меня этого недостаточно.
3297,ул үз бүлмәсендә.,он в своей комнате.
3298,ул үз бүлмәсендә.,он у себя в комнате.
3299,алга һәм җыр белән!,вперёд и с песней!
